In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import math
import pandas as pd 
import time
import numpy as np
import os
import wget

In [2]:
driver = webdriver.Chrome()

driver.get("http://www.instagram.com")

username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

username.clear()
username.send_keys("********")
password.clear()
password.send_keys("********")

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [3]:
time.sleep(5)
alert = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Не сейчас")]'))).click()
alert2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Не сейчас")]'))).click()

In [4]:
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Поиск']")))
searchbox.clear()

keyword = "#photo"
searchbox.send_keys(keyword)

time.sleep(5)
my_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/" + keyword[1:] + "/')]")))
my_link.click()

In [14]:
def get_amount_of_likes():
    try:
        likes = driver.execute_script('''return document.getElementsByClassName('Nm9Fw')[0].lastElementChild.getElementsByTagName('span')[0].innerText''')
        return likes
    except Exception:
        return np.nan

In [15]:
def get_date():
    try:
        date = driver.execute_script('''return document.getElementsByClassName('_1o9PC Nzb55')[0].getAttribute('datetime')''')
        return date
    except Exception:
        return np.nan

In [16]:
def get_author():
    try:
        author = driver.execute_script('''return document.getElementsByClassName('C4VMK')[0].getElementsByTagName('span')[0].innerText''')
        return author
    except Exception:
        return np.nan

In [17]:
def get_amount_of_subscribers():
    try:
        element_to_hover_over = driver.find_element_by_class_name("e1e1d")
        hover = ActionChains(driver).move_to_element(element_to_hover_over)
        hover.perform()
        time.sleep(2)
        follow = driver.execute_script('''return document.getElementsByClassName('g47SY lOXF2')[1].getAttribute('title')''')
        return follow
    except Exception:
        return np.nan

In [18]:
def get_tags():
    try:
        tags = driver.execute_script('''return document.getElementsByClassName(' xil3i')''')
        list = []
        for i in range(len(tags)):
            tag = driver.execute_script('''return document.getElementsByClassName(' xil3i')[{0}].innerText'''.format(i))
            list.append(tag)
        return list
    except Exception:
        return np.nan

In [19]:
def check_type():
    type = driver.find_element_by_xpath('//meta[@property="og:type"]').get_attribute('content')
    if type == 'image':
        return True
    else:
        return False

In [20]:
def next_post():
    try:
        time.sleep(0.5)
        element_to_hover_over = driver.find_element_by_class_name("_8-yf5 ")
        hover = ActionChains(driver).move_to_element(element_to_hover_over)
        hover.perform()
        time.sleep(0.5)
        next = driver.find_element_by_class_name("coreSpriteRightPaginationArrow")
        next.click()
        time.sleep(2)
    except:
        time.sleep(3)
        next_post()

In [21]:
def find_post(ref):
    try:
        driver.get(str(ref))
        time.sleep(1)
        img = driver.find_elements_by_tag_name('img')
        img = [i.get_attribute('src') for i in img]
        return img[1]
    except Exception:
        return np.nan       

In [22]:
def save_df():
    path_to_saxe_df = os.getcwd()
    path_to_saxe_df = os.path.join(path_to_saxe_df, "selfies.csv")
    df.to_csv(r'{0}'.format(path_to_saxe_df), index = False)

In [23]:
path = os.getcwd()
path = os.path.join(path, 'photoset_4500')

def save_picture(i):
    try:
        save_as = os.path.join(path, 'photo' + str(i) + '.jpg')
        wget.download(df['Download URL'][i], save_as)
    except Exception:
        df['Marker'][i] = 1  

In [24]:
###
def save_picture_now(reference):
    try:
        save_as = os.path.join(path, 'photo' + str(i) + '.jpg')
        wget.download(reference, save_as)
    except Exception:
        df['Marker'][i] = 1 

In [194]:
driver.get(str(df['Picture URL'][6]))
time.sleep(1)
img = driver.find_elements_by_tag_name('img')
img = [i.get_attribute('src') for i in img]
print(img[1])

https://scontent-arn2-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/175511835_271520651338806_5799219358781511657_n.jpg?tp=1&_nc_ht=scontent-arn2-1.cdninstagram.com&_nc_cat=1&_nc_ohc=PUA1wZq69SIAX972u9m&edm=APU89FAAAAAA&ccb=7-4&oh=bdca5168e8602b0a880906cb8f631259&oe=60A2E7A0&_nc_sid=86f79a


In [25]:
df = pd.DataFrame(columns = ['Picture URL' , 'Amount of likes', 'Author name', 'Amount of subscribers', 'Post tags', 'Post date', 'Download URL', 'Marker'])
#df

In [26]:
start_time = time.time()
num = 10


for i in range(num):
    if i % 100 == 0:
        print(i)
    URL = driver.current_url
    likes = get_amount_of_likes()
    name = get_author()
    subscribers = get_amount_of_subscribers()
    tags = get_tags()
    date = get_date()
    df = df.append({'Picture URL': URL, 'Amount of likes': likes, 'Author name': name,
        'Amount of subscribers': subscribers, 'Post tags': tags, 'Post date': date, 'Marker': 0}, ignore_index=True)
    next_post()

print("--- %s seconds ---" % (time.time() - start_time))
#10 sec

0
--- 59.50223970413208 seconds ---


In [28]:
df

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
0,https://www.instagram.com/p/COM6Uu5J7br/,190,hansekatzen,NaN,"[#hansekatzen, #bkh, #britischkurzhaar, #bsh, ...",2021-04-28T07:45:31.000Z,NaN,0
1,https://www.instagram.com/p/COMsermJWyE/,144,gingerthehooligan,3 359,"[#gingercat, #orangecat, #cat, #cats, #catsofi...",2021-04-28T05:44:32.000Z,NaN,0
2,https://www.instagram.com/p/COMhFPkJHWG/,1 530,cats_instagra_m,44 578,[],2021-04-28T04:04:57.000Z,NaN,0
3,https://www.instagram.com/p/COM00xrpfvM/,228,cocoandcarlo,6 023,"[#catsofinstagram, #bkh, #bsh, #bengalcat, #bk...",2021-04-28T06:57:28.000Z,NaN,0
4,https://www.instagram.com/p/COMl6Q9JVuY/,104,taksa_i_trikota,2 455,"[#munchkin, #munchkincat, #redcat, #sunnycat, ...",2021-04-28T04:47:08.000Z,NaN,0
5,https://www.instagram.com/p/CONKDdahCGC/,133,norachoto5656,7 413,"[#猫, #cat, #ねこ, #catstagram, #ねこすたぐらむ, #猫好きさんと...",2021-04-28T10:02:58.000Z,NaN,0
6,https://www.instagram.com/p/COMvJRoJ7vh/,240,thecharmingcats,13 075,"[#catphoto, #cats_of_instagram, #catloversday,...",2021-04-28T06:07:50.000Z,NaN,0
7,https://www.instagram.com/p/COMM3f2paJh/,1 109,kina_0818,NaN,"[#今日のきな🌻, #猫, #ねこすた, #ねこすきさんと繋がりたい, #ねこのいる生活,...",2021-04-28T01:08:19.000Z,NaN,0
8,https://www.instagram.com/p/COMsj0lJgmE/,165,finnishlifestylecat,7 080,"[#lifelessons, #lifequotes, #wisdom, #wisdomof...",2021-04-28T05:45:15.000Z,NaN,0
9,https://www.instagram.com/p/COM9EBBJIqp/,120,khaleesi_loki,2 377,"[#khaleesi, #loki, #russischblau, #russianblue...",2021-04-28T08:09:27.000Z,NaN,0


In [611]:
######## Download URL ########


In [418]:
df[df['Download URL'].isin([np.nan])]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker


In [748]:
df['Download URL'].replace(np.nan, 'x', inplace=True)
df['Download URL'].replace([None], 'x', inplace=True)

In [749]:
df[df['Download URL'].isin(['x'])]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
5,https://www.instagram.com/p/CN1rQ27sci0/,478,tortofea_valentina,NaN,"['#aврaл', '#уфаторт', '#тортназаказуфа', '#то...",2021-04-19T07:11:23.000Z,x,0
6,https://www.instagram.com/p/CN1cYNgn7ab/,533,elena_recep,6 395,"['#селфи', '#selfie', '#хорошеенастроение', '#...",2021-04-19T05:01:19.000Z,x,0
7,https://www.instagram.com/p/CNzjBMHnCIj/,1 904,barbie.001_,75 019,[],2021-04-18T11:20:51.000Z,x,0
8,https://www.instagram.com/p/CN2GEmRgEpT/,223,gentspeaker,1 502,"['#gentspeaker', '#гент', '#бельгия', '#фландр...",2021-04-19T11:05:38.000Z,x,0
9,https://www.instagram.com/p/CN15qwRAFln/,960,selenitii,11 937,[],2021-04-19T09:17:15.000Z,x,0
...,...,...,...,...,...,...,...,...
564,https://www.instagram.com/p/CN5B-ZNJ8-7/,104,goltsevababich,2 488,"['#мастерская', '#художник', '#дети', '#ученик...",2021-04-20T14:27:34.000Z,x,0
568,https://www.instagram.com/p/CN5Bu5gJx65/,35,kety_katrin,297,"['#друг', '#селфи', '#утродоброе']",2021-04-20T14:25:27.000Z,x,0
570,https://www.instagram.com/p/CN5BkjJrIsv/,10,sl.metlife,839,"['#страховка', '#пенсия', '#страхованиежизни',...",2021-04-20T14:24:02.000Z,x,0
571,https://www.instagram.com/p/CN5BbzIgpky/,26,7ingapavlova7,1 893,"['#павловаинга', '#фото', '#фотодень', '#весна...",2021-04-20T14:22:50.000Z,x,0


In [747]:
#fix Download URL missed values
for i in range (0, 574):
    #try:
        if (df['Download URL'].iloc)[i] == 'x':
            (df['Download URL'].iloc)[i] = find_post(df['Picture URL'][i])
        #else:
            #print(i)
    #except Exception:
        #print(i)

c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [510]:
df[3000:3050]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
3000,https://www.instagram.com/p/CNzUqmeHPm8/,28,handle_music,74,"['#фотограф', '#кружка', '#фотография', '#фото...",2021-04-18T09:15:26.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
3001,https://www.instagram.com/p/CN1OHdbjP8R/,12,beautiful_earth72,2,"['#beautiful_earth72', '#облака', '#горы', '#о...",2021-04-19T02:56:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3002,https://www.instagram.com/p/CN1NJEuq8wQ/,NaN,7felt_pen7,826,"['#инстаграм', '#инстаграманет', '#инстаграмне...",2021-04-19T02:56:10.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3003,https://www.instagram.com/p/CN1OCRAJEA9/,NaN,exz.tazy.muz,379,"['#инстаграм_порусски', '#я', '#любовь', '#инс...",2021-04-19T02:56:09.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3004,https://www.instagram.com/p/CN1N371Bm_y/,NaN,relaxx.muz.1,19,"['#инстаграм_порусски', '#я', '#любовь', '#инс...",2021-04-19T02:54:46.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3005,https://www.instagram.com/p/CN1NxnzpjVp/,6,foto_jam_oz,86,"['#jam_foto_oz', '#orehovozuevo', '#oz', '#ore...",2021-04-19T02:53:43.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3006,https://www.instagram.com/p/CN1Nu7InKni/,9,zarinaa1an,48,"['#лубимий', '#фото']",2021-04-19T02:53:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3007,https://www.instagram.com/p/CN1NqoBDOsu/,992,frolovaskakovskaia,8 495,"['#girl', '#beauty', '#cute', '#streetstyle', ...",2021-04-19T02:52:45.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3008,https://www.instagram.com/p/CN1NpXSg4oN/,9,sveistr,103,"['#небо', '#раннееутро', '#заря', '#отражение'...",2021-04-19T02:52:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3009,https://www.instagram.com/p/CN1NpPsM8a7/,17,handmade_elv,937,"['#обложканапаспорт', '#органайзер', '#уфа', '...",2021-04-19T02:52:34.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0


In [580]:
df[df['Amount of subscribers'].isin([np.nan])]


,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker


In [ ]:
######## Amount of subscribers ########

In [722]:
df['Amount of subscribers'].replace('x', np.nan, inplace=True)

In [684]:
df['Amount of subscribers'].replace(np.nan, 'x', inplace=True)
df['Amount of subscribers'].replace([None], 'x', inplace=True)

In [723]:
len(df[df['Amount of subscribers'].isin(['x'])])

0

In [686]:
df['Amount of subscribers'][72]

'762'

In [724]:
df[df['Amount of subscribers'].isin([np.nan])]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
127,https://www.instagram.com/p/CNzTPv_r0kn/,NaN,NaN,NaN,[],NaN,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
388,https://www.instagram.com/p/CNzR_BoBjHn/,2,mrm_trends,NaN,"['#l', '#girl', '#likeforfollow', '#f', '#like...",2021-04-18T08:52:01.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
702,https://www.instagram.com/p/CNzRqMBAzYM/,NaN,milktine_,NaN,"['#selfie', '#summervibes', '#summer']",2021-04-18T08:49:10.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
900,https://www.instagram.com/p/CNzRbEtFPHa/,22,j__in_._,NaN,[],2021-04-18T08:47:06.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1323,https://www.instagram.com/p/CN1lwhwDBZ7/,3,che.nomazof,NaN,"['#rus_atmos', '#rus', '#Балаково', '#Balakovo...",2021-04-19T06:23:17.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
1327,https://www.instagram.com/p/CN1luY1DQQe/,4,che.nomazof,NaN,"['#природа', '#rus', '#Балаково', '#Balakovo',...",2021-04-19T06:22:59.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1330,https://www.instagram.com/p/CN1ls4MDxc5/,6,che.nomazof,NaN,"['#rus_atmos', '#rus', '#Балаково', '#Balakovo...",2021-04-19T06:22:47.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1347,https://www.instagram.com/p/CN1ketirImL/,NaN,NaN,NaN,[],NaN,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1391,https://www.instagram.com/p/CN1lMsVJ4_h/,NaN,NaN,NaN,[],NaN,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1591,https://www.instagram.com/p/CN1jeIpl3eh/,9,wwww.ke,NaN,"['#фото', '#фотомосква', '#фотосессия', '#двор...",2021-04-19T06:03:17.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0


In [651]:
for i in range (0, 4500):
        if (df['Amount of subscribers'].iloc)[i] == 'x':
            driver.get((df['Picture URL'].iloc)[i])
            (df['Amount of subscribers'].iloc)[i] = get_amount_of_subscribers()

c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
######## Amount of likes ########

In [715]:
df['Amount of likes'].replace('x', np.nan, inplace=True)

In [666]:
df['Amount of likes'].replace(np.nan, 'x', inplace=True)
df['Amount of likes'].replace([None], 'x', inplace=True)

In [667]:
df['Amount of likes'][65]

'5'

In [716]:
df[df['Amount of likes'].isin(['x'])]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker


In [717]:
df[df['Amount of likes'].isin([np.nan])]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
127,https://www.instagram.com/p/CNzTPv_r0kn/,NaN,NaN,x,[],NaN,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
157,https://www.instagram.com/p/CNzSLarFXRI/,NaN,indianmemegram,903,"['#indianmemegram', '#today', '#love', '#insta...",2021-04-18T08:54:25.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
165,https://www.instagram.com/p/CNzSCX5j-Ov/,NaN,gayxvibez,1 892,"['#gay', '#instagay', '#lgbt', '#gayboy', '#lg...",2021-04-18T08:54:16.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
186,https://www.instagram.com/p/CNzSCHOgoDM/,NaN,selhoztehnika.ru,83,"['#сельхозтехника', '#трактор', '#трактористы'...",2021-04-18T08:54:09.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
190,https://www.instagram.com/p/CNzSE-hjbjX/,NaN,mizahi.duygu,150,"['#friends', '#smile', '#instagood', '#life', ...",2021-04-18T08:54:07.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...,...
4440,https://www.instagram.com/p/CN2IAgXhO6t/,NaN,shoyebahammed,8 729,"['#love', '#followback', '#instagramers', '#me...",2021-04-19T11:23:06.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
4449,https://www.instagram.com/p/CMzwCqsjP-Z/,NaN,mohamedaminefarh,1 497,"['#love', '#art', '#repost', '#tbt', '#travel'...",2021-03-24T16:50:50.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
4472,https://www.instagram.com/p/CN2H_CJBmtp/,NaN,mohamed_elmstkwy,379,"['#egypt', '#Cairo', '#white_desert', '#Sahara...",2021-04-19T11:22:49.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
4479,https://www.instagram.com/p/CN2IB64r3gW/,NaN,ghazal.wedding.planner,950,"['#كنب', '#تاجيركنب', '#تأجير_كنب', '#غنفات',...",2021-04-19T11:22:45.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0


In [665]:
for i in range (0, 4500):
        if (df['Amount of likes'].iloc)[i] == 'x':
            driver.get((df['Picture URL'].iloc)[i])
            (df['Amount of likes'].iloc)[i] = get_amount_of_likes()

c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


KeyboardInterrupt: 

In [ ]:
######## Author name ########

In [711]:
df['Author name'].replace('x', np.nan, inplace=True)

In [678]:
df['Author name'].replace(np.nan, 'x', inplace=True)
df['Author name'].replace([None], 'x', inplace=True)

In [679]:
df['Author name'][40]

'x'

In [712]:
df[df['Author name'].isin(['x'])]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker


In [713]:
df[df['Author name'].isin([np.nan])]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
40,https://www.instagram.com/p/CNzTROkJTDv/,7,NaN,82,[],2021-04-18T09:03:14.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
127,https://www.instagram.com/p/CNzTPv_r0kn/,x,NaN,x,[],NaN,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
171,https://www.instagram.com/p/CNzSPCzlSIj/,22,NaN,1 421,[],2021-04-18T08:54:12.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
233,https://www.instagram.com/p/CNzOe0IBiyE/,111,NaN,689,[],2021-04-18T08:21:24.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
723,https://www.instagram.com/p/CNy4mkBF_W6/,240,NaN,2 647,[],2021-04-18T05:10:13.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
817,https://www.instagram.com/p/CNruQB4nOru/,122,NaN,596,[],2021-04-15T10:25:05.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
878,https://www.instagram.com/p/CNrumXdHm_1/,131,NaN,596,[],2021-04-15T10:28:08.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
991,https://www.instagram.com/p/CNxRRwlLJpM/,149,NaN,244,[],2021-04-17T14:07:21.000Z,https://www.instagram.com/static/images/web/mo...,0
1015,https://www.instagram.com/p/CNzRRcjjnHN/,202,NaN,1 133,[],2021-04-18T08:45:47.000Z,https://www.instagram.com/static/images/web/mo...,0
1122,https://www.instagram.com/p/CNzROPMHGFp/,22,NaN,15,[],2021-04-18T08:45:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0


In [677]:
for i in range (0, 4500):
        if (df['Author name'].iloc)[i] == 'x':
            driver.get((df['Picture URL'].iloc)[i])
            (df['Author name'].iloc)[i] = get_author()

c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
#####################

In [ ]:
start_time = time.time()
for i in range(3488, 3498):    
    (df['Download URL'].iloc)[i] = find_post(df['Picture URL'][i])
print("--- %s seconds ---" % (time.time() - start_time))

In [730]:
start_time = time.time()
for i in range(len(df)):
    save_picture(i)
print("--- %s seconds ---" % (time.time() - start_time))

  0% [                                                                            ]      0 / 495434

c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


100% [............................................................................] 124125 / 124125--- 770.6910183429718 seconds ---


In [735]:
path_to_download = os.getcwd()
path_to_download = os.path.join(path_to_download, 'data')
path_to_download = os.path.join(path_to_download, 'photos_575.csv')
df = pd.read_csv(path_to_download)

In [729]:
save_df()

In [ ]:
######## Delete raws ########

In [728]:
df = df[df['Picture URL'].notna()]
df = df[df['Amount of likes'].notna()]
df = df[df['Author name'].notna()]
df = df[df['Amount of subscribers'].notna()]
df.reset_index(inplace=True, drop=True) 
df


,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
0,https://www.instagram.com/p/CNzI3m9MNxO/,165,romina_stanescu,1 397,"['#morningselfie', '#morningmood', '#selfie']",2021-04-18T07:32:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1,https://www.instagram.com/p/CNzKR83HAvI/,1 217,yeonions,299 936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
2,https://www.instagram.com/p/CNzCpoGn1_U/,1 441,hyo._.jin23,63 119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
3,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10 961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
4,https://www.instagram.com/p/CNzOicxHVOT/,1 479,all_zero._.k,104 242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...,...
3946,https://www.instagram.com/p/CN2IA8Jnr4C/,828,dislav_v,8 403,"['#in', '#instagram', '#insta', '#lol', '#1', ...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3947,https://www.instagram.com/p/CN2IAvtFWZW/,31,joshuabravo,765,"['#mensfashion', '#fashion', '#style', '#mensw...",2021-04-19T11:22:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3948,https://www.instagram.com/p/CN2IAYisu4A/,38,mala.mi.trenuje,1 411,"['#rozRUSZAMY', '#polishgirl', '#fitgirl', '#f...",2021-04-19T11:22:32.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3949,https://www.instagram.com/p/CN2IAS-jrh-/,33,tushargedar,360,"['#likes', '#like', '#follow', '#likeforlikes'...",2021-04-19T11:22:32.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0


In [734]:
df[1738:1780]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
1738,https://www.instagram.com/p/CN1go9rhEp1/,46,ka_mesha4,85,"['#красиво', '#лайктайм', '#лайкивзаимно', '#l...",2021-04-19T05:38:33.000Z,https://scontent-frt3-1.cdninstagram.com/v/t51...,0
1739,https://www.instagram.com/p/CN1goHrJtY6/,44,andreyslepkov_photo,2 591,"['#мобильноефото', '#фотосессии', '#фотографво...",2021-04-19T05:38:26.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1740,https://www.instagram.com/p/CN1glIPgatP/,6,_zoppppp_456,66,"['#фото', '#фотография', '#луна', '#закат', '#...",2021-04-19T05:38:02.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1741,https://www.instagram.com/p/CN1gkxzJtRW/,53,orenmassage56,4 217,"['#оксанамассаж', '#оксанамиллер', '#девушкиор...",2021-04-19T05:37:59.000Z,https://scontent-frt3-1.cdninstagram.com/v/t51...,0
1742,https://www.instagram.com/p/CN1gkrrjFv6/,14,irina_bloger_moskva,1 090,"['#подснежник', '#фото', '#selfi', '#весна', '...",2021-04-19T05:37:58.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
1743,https://www.instagram.com/p/CN1gkMiLHdy/,18,lider_nrd_life_,598,"['#тренер', '#чемпион', '#мастер', '#опыт', '#...",2021-04-19T05:37:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1744,https://www.instagram.com/p/CN1gipfln8Y/,27,ishchuknastya,154,"['#фотографищуканастасия', '#фотографищукнастя...",2021-04-19T05:37:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1745,https://www.instagram.com/p/CN1ghzQnb0E/,9,____mi_ho____,539,"['#новокузнецк', '#стихисобственногосочинения'...",2021-04-19T05:37:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1746,https://www.instagram.com/p/CN1ghjqMrBN/,6,albertsochinetz,983,"['#plastic', '#bag', '#jellyfish', '#garbage',...",2021-04-19T05:37:33.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
1747,https://www.instagram.com/p/CN1ggWAgVA4/,72,nutka_pichienko,2 975,"['#книги', '#материнство', '#круассаны', '#клу...",2021-04-19T05:37:23.000Z,https://scontent-frt3-1.cdninstagram.com/v/t51...,0


In [ ]:
# emulate click on a likes button
#driver.execute_script('''document.getElementsByClassName('Nm9Fw')[0].lastElementChild.click()''')

# scroll down by 4000px to load more users who liked the post
#driver.execute_script('''document.getElementsByClassName('Igw0E IwRSH eGOV_ vwCYk i0EQd')[0].firstChild.scrollBy(0, 4000);''')
#time.sleep(0.2)

# collect usernames
#usernames = driver.execute_script('''return document.getElementsByClassName('_7UhW9 xLCgt MMzan KV-D4 fDxYl');''')

# each username sits inside of a div, so to get actual username we can use .get_attribute() method
#usernames[0].get_attribute('innerText')